In [1]:
from mqt.yaqs.core.data_structures.networks import MPS, MPO
from mqt.yaqs import simulator
from mqt.yaqs.core.data_structures.simulation_parameters import AnalogSimParams, Observable
from mqt.yaqs.core.libraries.gate_library import X, Z, RuntimeCost
from mqt.yaqs.core.data_structures.noise_model import NoiseModel

import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
def tdvp_simulator(H_0, noise_model, state=None):
    L = H_0.length

    # Ising
    state = MPS(length=L)

    # Heisenberg
    # elapsed_time = 2
    # state = MPS(length=L, state="Neel")

    measurements = [Observable(Z(), [0])] + [Observable("entropy", [L//2, L//2+1])] + [Observable("max_bond")]
    sim_params = AnalogSimParams(observables=measurements,
                                elapsed_time=5,
                                dt=0.1,
                                num_traj=10,
                                threshold=1e-3,
                                trunc_mode="discarded_weight",
                                order=2,
                                sample_timesteps=False)

    simulator.run(state, H_0, sim_params, noise_model=noise_model)
    print("Obs Exp Val", sim_params.observables[0].results[-1])
    print("Entropy", sim_params.observables[1].results[-1])
    print("Max Bond", sim_params.observables[2].results[-1])
    return sim_params.observables[1::]

In [ ]:
L_list = [8, 12, 16, 20, 24, 28, 32, 40, 48, 64]
for L in L_list:
    print(L)
    J = 1
    h = 1
    H_0 = MPO()
    H_0.init_ising(L, J, h)

    dps = np.logspace(-3, 0, 10)
    print(dps)
    results = []
    for j, dp in enumerate(dps):
        print(j+1, "of", len(dps))
        gamma = dp/0.1  # assuming dt=0.1
        # Unraveling 1
        noise_model = NoiseModel([
            {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["pauli_z", "pauli_x", "pauli_y"]
            ])
        # Unraveling 2
        # noise_model = NoiseModel([
        #     {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["measure_0", "measure_1"]
        # ])
        cost = tdvp_simulator(H_0, noise_model)
        results.append(cost)

    filename = f"heatmap_{L}.pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(results, handle)

8
[0.001      0.00215443 0.00464159 0.01       0.02154435 0.04641589
 0.1        0.21544347 0.46415888 1.        ]
1 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:05<00:00,  1.72it/s]


Obs Exp Val (-0.14769782549904362+0j)
Entropy (0.5126118366542388+0j)
Max Bond (2+0j)
2 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:06<00:00,  1.67it/s]


Obs Exp Val (0.03361192364135964+0j)
Entropy (0.532093449468089+0j)
Max Bond (2+0j)
3 of 10


Running trajectories:   0%|                              | 0/10 [00:00<?, ?it/s]